In [15]:
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,"
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,"
    "org.postgresql:postgresql:42.5.0 pyspark-shell"
)

In [13]:
import pandas as pd

# df = pd.read_csv("/home/lupusruber/music_analytics/dummy_dwh/SongDim.csv")

In [17]:
# df.groupby("SongSK").size().reset_index(name="count")["count"].unique()